In [59]:
from collections import defaultdict

In [68]:
def build_full_length_alignments(sam_path, ref_fasta_path):
    """
    Efficiently builds fully gapped reference-anchored alignments.
    Insertions relative to reference create new columns; deletions are '-'.
    """
    from collections import defaultdict

    # Load reference
    with open(ref_fasta_path) as f:
        ref = "".join(line.strip() for line in f if not line.startswith(">")).upper()
    ref_len = len(ref)

    # List to hold each read as list of (ref_pos, base) tuples
    reads_data = []

    # Dictionary: ref_pos -> list of inserted bases at this position (to determine columns)
    insertions_at_pos = defaultdict(list)

    # Parse SAM
    with open(sam_path) as sam:
        for line in sam:
            if line.startswith("@"):
                continue
            fields = line.strip().split("\t")
            flag = int(fields[1])
            if flag & 4:  # unmapped
                continue

            pos = int(fields[3]) - 1
            cigar = fields[5]
            seq = fields[9].upper()
            if cigar == "*":
                continue

            ref_i = pos
            read_i = 0
            num = ""
            read_tuples = []  # list of (ref_pos, base)

            for char in cigar:
                if char.isdigit():
                    num += char
                    continue
                length = int(num)
                num = ""

                if char in "M=X":
                    for _ in range(length):
                        read_tuples.append((ref_i, seq[read_i]))
                        ref_i += 1
                        read_i += 1
                elif char == "D":
                    for _ in range(length):
                        read_tuples.append((ref_i, "-"))
                        ref_i += 1
                elif char == "I":
                    ins_seq = seq[read_i:read_i+length]
                    insertions_at_pos[ref_i].append(ins_seq)
                    read_tuples.append((ref_i, ins_seq))  # mark as insertion for later
                    read_i += length
                elif char == "S":
                    read_i += length
                elif char == "H":
                    pass

            reads_data.append(read_tuples)

    # ---------------------------------------------------------
    # Determine final alignment length
    # ---------------------------------------------------------
    # For each reference position, compute max number of insertions
    max_inserts = {}
    for pos, ins_lists in insertions_at_pos.items():
        max_len = max(len(ins_seq) for ins_seq in ins_lists)
        max_inserts[pos] = max_len

    # Compute total alignment length
    alignment_length = ref_len + sum(max_inserts.values())

    # Map reference positions to final alignment positions
    ref_to_aln = {}
    aln_pos = 0
    for i in range(ref_len):
        ref_to_aln[i] = aln_pos
        aln_pos += 1
        if i in max_inserts:
            aln_pos += max_inserts[i]

    # ---------------------------------------------------------
    # Build reference row
    # ---------------------------------------------------------
    ref_row = ["-"] * alignment_length
    for i, base in enumerate(ref):
        ref_row[ref_to_aln[i]] = base

    # ---------------------------------------------------------
    # Build each read row
    # ---------------------------------------------------------
    read_rows = []
    for read_tuples in reads_data:
        row = ["-"] * alignment_length
        for ref_pos, base in read_tuples:
            aln_idx = ref_to_aln[ref_pos]
            if isinstance(base, str) and len(base) == 1:
                row[aln_idx] = base
            else:  # insertion string
                # Place inserted bases after the reference column
                for j, b in enumerate(base):
                    row[aln_idx + j] = b
        read_rows.append(row)

    # Convert to strings
    ref_str = "".join(ref_row)
    subread_strings = ["".join(r) for r in read_rows]

    return ref_str, subread_strings


In [69]:
ref_aln, subread_strings = build_full_length_alignments('erap_frags_aligned.sam', 'erap_reference.fasta')

In [70]:
'-' in ref_aln

True

In [71]:
len(ref_aln), len(subread_strings[0])

(48764, 48764)

In [73]:
def print_aln(ref, query):
    for i in range(0, len(ref), 80):
        print(ref[i:i+80], query[i:i+80], sep='\n', end='\n\n')


In [74]:
from collections import Counter

In [75]:
erap_reference = '...'

In [76]:
def add_pipes(piped_reference, ref_aln, read_alns):

    print(piped_reference[:10], ref_aln[:10])

    read_count = len(read_alns) 

    piped_ref_aln = []
    alns_transposed = []

    ref_iterator = iter(erap_reference)
    
    for ref_ali_base, read_ali_bases in zip(
        ref_aln,
        zip(*read_alns)
    ):
        if ref_ali_base != '-':
            ref_base = next(ref_iterator)
            if ref_base == '|':
                piped_ref_aln.append('|')
                alns_transposed.append(['|'] * read_count)
                ref_base = next(ref_iterator)
            piped_ref_aln.append(ref_ali_base)
            alns_transposed.append(read_ali_bases)
        else:
            piped_ref_aln.append(ref_ali_base)
            alns_transposed.append(read_ali_bases)

    print(len(piped_ref_aln))
    print(len(alns_transposed))

    piped_ref_aln = ''.join(piped_ref_aln)
    piped_read_alns = [
        ''.join(bases)
        for bases
        in zip(*alns_transposed)
    ]

    return piped_ref_aln, piped_read_alns
            

piped_ref_aln, piped_read_alns = add_pipes(erap_reference, ref_aln, subread_strings)

GTACAGTGGC GTACAGTGGC
48804
48804


In [77]:
print_aln(piped_ref_aln, piped_read_alns[100])

GTACAGTGGCCCTTGGTAGTGCAGGAAAGTCCTGGGGGCCACCTCTAACCCACCTTCCTCCTCTACAGCATCTCCCACTG
------------------------GAAAGTCCTGGGGGCCACCTCTAACCCACCTTCCTCCTCTACAGCATCTCCCACTG

TAGTCATTCTCTACCGAAGCCCCAGAAGGTGCGGCACTTTGCCACGACAGAGTACTGGGTTCATGTTTCTTTCCGAGGCG
TAGTCATTCTCTACCGAAGCCCCAGAAGGTGCGGCACTTTGCCACGACAGAGTACTGGGTTCATGTTTCTTTCCGAGGCG

GGCCAAGAGCTCTCAGCCCACTGGCAGTGGCGAGATGACGGACACCCAGCGAGTCCAATGGGCGTCGAACGCGTCTAGGC
GGCCAAGAGCTCTCAGCCCACTGGCAGTGGCGAGATGACGGACACCCAGCGAGTCCAATGGGCGTCGAACGCGTCTAGGC

TTGGTGGACTTGTCAGCGCCTGCCTGGCTTCGGTCCCCAACTTGAGCACCGGCCCTTTCCTGCATGCCCCTAACCCTCGC
TTGGTGGACTTGTCAGCGCCTGCCTGGCTTCGGTCCCCAACTTGAGCACCGGCCCTTTCCTGCATGCCCCTAACCCTCGC

AACGCTAAACAGTGA-AAAAAAAAAAAAGACAAAAACAAAAAGCATCTCAA--------------------CACACACAC
AACGCTAAACAGTGA-AAAAAAAAAAAAGACAAAAACAAAAAGCATCTCAACACACACACACACACACAC-CACACACAC

ACACACACACACACACACACACACACACACACACACACACACACACACACACACACACACACACGGATCCGCGTTCAGAA
ACACACACACACACACACACACACACACACACACACACACACACACACACACACACACACACACGGATCCGCGTTCAGAA

AGGCGTGCACTTCCTACGCCTG

In [56]:
print('-' in piped_ref_aln)

False
